In [ ]:
import numpy as np
import random
import mbuild as mb
from mbuild.lib.recipes.polymer import Polymer
from mbuild.coordinate_transform import force_overlap
from mbuild.lib.atoms import H
from mbuild.port import Port
from foyer import Forcefield
from openbabel import openbabel as ob

In [ ]:
#Read the monomers that make up the polymer
block1_smiles_list = []
total_charge_sum = 0.0
with open('mol.txt', 'r') as f:
    for i, line in enumerate(f, start=1):
        if 1 <= i <= 6:
            block1_smiles_list.append(line.strip())
block2_smiles = 'C'
repeat_units_block1 = 50

In [ ]:
#Convert from top to itp
def getItpFromTop(top, itp, molname):
    print("Processing", top)
    with open(top, "r") as f:
        lines = [line.rstrip('\n') for line in f]
    nLmoleculetype, nLatoms, nLbonds, nLsystem = -1, -1, -1, -1

    for i, line in enumerate(lines):
        if "[ moleculetype ]" in line:
            nLmoleculetype = i
        elif "[ atoms ]" in line:
            nLatoms = i
        elif "[ bonds ]" in line:
            nLbonds = i
        elif "[ system ]" in line:
            nLsystem = i

    if -1 in (nLmoleculetype, nLatoms, nLbonds, nLsystem):
        raise ValueError("One or more required sections missing in the .top file!")

    print(f"Located sections: moleculetype={nLmoleculetype}, atoms={nLatoms}, bonds={nLbonds}, system={nLsystem}")

    lines[nLmoleculetype + 2] = f"{molname}          3"

    with open(itp, "w") as f:
        atmidx = 0
        for i, line in enumerate(lines[nLmoleculetype:nLsystem]):
            if nLatoms < i + nLmoleculetype < nLbonds:
                if len(line.strip()) > 0 and not line.startswith(";"):
                    dat = line.split()
                    if dat[3] == "RES":
                        dat[3] = molname
                    line = " ".join(dat[:8]) 
                    atmidx += 1
            f.write(line + '\n')

    print("ITP file generation complete.")


In [ ]:
molname = "mol{}".format(i)
for i, block1_smiles in enumerate(block1_smiles_list, start=1):
    # Generate polymer_smiles by repeating block1_smiles and end Group
    polymer_smiles = block1_smiles * repeat_units_block1 + block2_smiles
    mol = mb.load(polymer_smiles, smiles=True, backend='pybel')

    ff = Forcefield(forcefield_files='./oplsaaXian.xml')
    obConversion = ob.OBConversion()
    mol_ob = ob.OBMol()
    ob_method = "eem"
    ob_charge_model = ob.OBChargeModel.FindType(ob_method)
    mol.save(f'./mol{i}.mol2', overwrite=True)
    mol.save(f'./mol{i}.pdb', overwrite=True)

    mol_top = ff.apply(mol, verbose=True)
    obConversion.ReadFile(mol_ob, f'./mol{i}.pdb')
    ob_charge_model.ComputeCharges(mol_ob)
    charges = ob_charge_model.GetPartialCharges()

    total_charge_sum = 0
    for atom, charge in zip(mol_top.atoms, charges):
        atom.charge = charge
        total_charge_sum += charge
    print(f"Total charge sum for molecule {i}: {total_charge_sum}")

    mol_top.save(f'./mol{i}.top', overwrite=True)

    #Generate .itp files for each polymer
    molname = f"mol{i}"
    getItpFromTop(f'./mol{i}.top', f'./mol{i}.itp', molname)